In [7]:
# import necessary libraries
import requests
from bs4 import BeautifulSoup as bs

# specify the parameters of the browser settings
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36'}

# create a url list to scrape data from all pages
html = requests.get('https://honolulu.craigslist.org/d/apartments-housing-for-rent/search/apa', headers = header)
html.status_code

200

In [2]:
# it is important to get the time of pauses that we will do during the scraping to avoid getting thrown from the server
from time import *
from random import randint


for i in range(0,3):
  # selects random integer in given range
  x = randint(2,5)
  print(x)
  sleep(x)
  print(f'I waited {x} seconds')

2
I waited 2 seconds
2
I waited 2 seconds
4
I waited 4 seconds


In [3]:
# get the content of the base page
page = requests.get("https://honolulu.craigslist.org/d/apartments-housing-for-rent/search/apa", headers = header)
bsobj = soup(page.content,'lxml')
bsobj

<!DOCTYPE html>
<html class="no-js">
<head>
<meta charset="utf-8"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<meta content="craigslist" property="og:site_name"/>
<meta content="preview" name="twitter:card"/>
<meta content="hawaii apartments / housing for rent - craigslist" property="og:title"/>
<meta content="hawaii apartments / housing for rent - craigslist" name="description"/>
<meta content="hawaii apartments / housing for rent - craigslist" property="og:description"/>
<meta content="https://honolulu.craigslist.org/d/apartments-housing-for-rent/search/apa" property="og:url"/>
<title>hawaii apartments / housing for rent - craigslist</title>
<link href="https://honolulu.craigslist.org/d/apartments-housing-for-rent/search/apa" rel="canonical"/>
<link href="https://honolulu.craigslist.org/d/apartments-housing-for-rent/search/apa?s=120" rel="next"/>
<script id="ld_breadcrumb_data" type="application/ld+json

In [4]:
# collect the links from the each craiglists page
links = []
for link in bsobj.findAll('h3',{'class':'result-heading'}):
    links.append(link.a['href'])

In [15]:

url = 'https://honolulu.craigslist.org/d/apartments-housing-for-rent/search/apa'

# function to get the page status the the bs object
def get_page(url):
    
    page = requests.get(url)
    status = page.status_code
    soup = bs(page.text, 'lxml')
    
    return [soup, status]


soup = get_page(url)[0]
print(soup)





<!DOCTYPE html>
<html class="no-js">
<head>
<meta charset="utf-8"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<meta content="craigslist" property="og:site_name"/>
<meta content="preview" name="twitter:card"/>
<meta content="hawaii apartments / housing for rent - craigslist" property="og:title"/>
<meta content="hawaii apartments / housing for rent - craigslist" name="description"/>
<meta content="hawaii apartments / housing for rent - craigslist" property="og:description"/>
<meta content="https://honolulu.craigslist.org/d/apartments-housing-for-rent/search/apa" property="og:url"/>
<title>hawaii apartments / housing for rent - craigslist</title>
<link href="https://honolulu.craigslist.org/d/apartments-housing-for-rent/search/apa" rel="canonical"/>
<link href="https://honolulu.craigslist.org/d/apartments-housing-for-rent/search/apa?s=120" rel="next"/>
<script id="ld_breadcrumb_data" type="application/ld+json

In [9]:
# collect the links fron the individual webpages of Craiglist
def get_links(soup):

    list_links = []
    for link in soup.findAll('h3',{'class':'result-heading'}):
        list_links.append(link.a['href'])
    
    return list_links

list_links = get_links(soup)
list_links

['https://honolulu.craigslist.org/oah/apa/d/honolulu-bed-2bath-in-the-heart-of-town/7359517025.html',
 'https://honolulu.craigslist.org/oah/apa/d/honolulu-studio-for-rent/7359515427.html',
 'https://honolulu.craigslist.org/oah/apa/d/ewa-beach-three-bedroom-townhome-in/7359504594.html',
 'https://honolulu.craigslist.org/big/apa/d/hilo-hilo-room-for-rent/7356635440.html',
 'https://honolulu.craigslist.org/oah/apa/d/honolulu-rarely-available-newer/7359497185.html',
 'https://honolulu.craigslist.org/kau/apa/d/lawai-lawai-bedroom-unit/7354289744.html',
 'https://honolulu.craigslist.org/oah/apa/d/honolulu-1st-class-condo-close-alana/7359490755.html',
 'https://honolulu.craigslist.org/oah/apa/d/kapolei-everything-you-need-all-right/7357619824.html',
 'https://honolulu.craigslist.org/oah/apa/d/honolulu-ocean-views-breezy-lanai-45th/7359490873.html',
 'https://honolulu.craigslist.org/oah/apa/d/honolulu-patio-balcony-in-select-units/7351614842.html',
 'https://honolulu.craigslist.org/oah/apa/d/h

In [12]:
# extract info from the webpages
get_info = []

def extract_info(list_links):
    
    # iterate across the saved links 
    for link in list_links:

        res = requests.get(link).text

        soup_ind_link = bs(res, 'lxml')
        
        # some issues were observed while extracting the coordinates,
        # thus I must declare a variable corr = [] before the scraping
        coor = []
        
        
        # try except is needed because sometimes the scraper might not find the data
        # or the data will have a different format or errors would occur
        try:
            
            area = soup_ind_link.find('span',{'class':'housing'}).text.strip()
            description = soup_ind_link.find('span',{'id':'titletextonly'}).text.strip()
            price = soup_ind_link.find('span',{'class':'price'}).text.strip()
            adress = soup_ind_link.find('span',{'class':'postingtitletext'}).small.text.strip()
            info = soup_ind_link.find('section',{'id':'postingbody'}).text.strip()
            post_date = soup_ind_link.find('div',{'class':'reply-button-row'}).text.strip().split()[11]
            add_info = soup_ind_link.find('p',{'class':'attrgroup'}).text.strip()
            allowed = soup_ind_link.find('div',{'class':'mapAndAttrs'}).text.strip().split()[12:]
            coor = soup_ind_link.find('p',{'class':'mapaddress'}).a['href']
            
        # Whatever info was not scraped it will be declared as None and these missing values we will have to 
        # impute later
        except:
            pass


        data = {'area': area, 
                'description': description,
                'price': price,
               'adress': adress,
                'info': info,
                'post_date': post_date,
                'add_info': add_info,
                'allowed': allowed,
                'coordinates': coor
                
               }
        
        get_info.append(data)
        
    return get_info

In [13]:

pg = 0

# number of pages of the rental prices (at the bottom of the Craiglist interface)
# you can put here 2801 as I did initially to get the full dataframe.
# I started with 280 just for demonstration purposes

while pg<280: 


    if pg == 0:
        url = f"https://honolulu.craigslist.org/d/apartments-housing-for-rent/search/apa?s"
        soup_status = get_page(url)
    else:
        url = f"https://honolulu.craigslist.org/d/apartments-housing-for-rent/search/apa?s={pg}"
    
    if soup_status[1] == 200: # valid response
        

        print(f'scraping page {pg}')
        sleep(randint(5,8)) # to avoid getting banned use the random timer to get to the page
        extract_info(get_links(soup_status[0]))
        pg += 120
        
    else:
        print('The end')
        break
        
        
        
df = pd.DataFrame(get_info)

scraping page 0
scraping page 120
scraping page 240


<IPython.core.display.Javascript object>

In [14]:
df

,area,description,price,adress,info,post_date,add_info,allowed,coordinates
0,/ 4br - 1058ft2 -,"4 bed-2bath In the heart of town, near bus st...","$2,595",(Honolulu),QR Code Link to This Post\n\n\nWaena Apartment...,2021-08-01,4BR / 2Ba\n1058ft2\navailable aug 18,"[dates, sunday, 2021-08-01, monday, 2021-08-02...",[]
1,/ 1br -,Studio for rent,"$1,500",(Honolulu),QR Code Link to This Post\n\n\nFully furnished...,2021-08-01,1BR / splitBa\navailable aug 1,"[house, dates, tuesday, 2021-08-03, tuesday, 2...","https://www.google.com/maps/search/21.295577,-..."
2,/ 3br - 1368ft2 -,Three Bedroom Townhome In Kapolei! Pet Friendl...,"$3,160",(Kapolei),QR Code Link to This Post\n\n\nCall Now - sho...,2021-08-01,3BR / 1.5Ba\n1368ft2,"[are, OK, -, wooof, flooring:, wood, townhouse...","https://www.google.com/maps/search/21.326765,-..."
3,/ 3br - 1368ft2 -,Three Bedroom Townhome In Kapolei! Pet Friendl...,"$3,160",(Kapolei),QR Code Link to This Post\n\n\nCall Now - sho...,2021-08-01,3BR / 1.5Ba\n1368ft2,"[are, OK, -, wooof, flooring:, wood, townhouse...",[]
4,/ 1br -,Rarely available Newer condominium + One of th...,$900,(Kapolei),QR Code Link to This Post\n\n\nCall Now - sho...,2021-08-01,3BR / 1.5Ba\n1368ft2,"[are, OK, -, wooof, flooring:, wood, townhouse...",[]
...,...,...,...,...,...,...,...,...,...
355,/ 400ft2 -,!!!Lg. Studio Kailua/ utilities included/2 blo...,"$1,100",(Kailua),QR Code Link to This Post\n\n\nLg studio furni...,2021-07-19,0BR / 1Ba\n400ft2,"[period:, yearly]",[]
356,/ 3br - 980ft2 -,"Single Family Home w/PV in Palama: 3 Bedrooms,...","$1,800",(Palama),QR Code Link to This Post\n\n\nRental Address:...,2021-07-26,3BR / 1Ba\n980ft2\navailable aug 1,"[980ft2, available, aug, 1, flooring:, tile, h...","https://www.google.com/maps/search/21.323319,-..."
357,/ 2br - 802ft2 -,"MOUNTAIN VIEW TERRACE: 2 Bedroom, 1 Bath, 1 Pa...","$1,950",(Kaneohe),QR Code Link to This Post\n\n\nRental Address:...,2021-07-20,2BR / 1Ba\n802ft2\navailable aug 9,"[w/d, in, unit, no, smoking, attached, garage]",[]
358,/ 3br - 1368ft2 -,SEASCAPE AT MAKAKILO: Single Family Home in Ga...,"$3,200",(Makakilo / Kapolei),QR Code Link to This Post\n\n\nRental Address:...,2021-07-26,3BR / 2.5Ba\n1368ft2\navailable aug 1,"[w/d, in, unit, no, smoking, attached, garage]",[]


In [ ]:
df.to_csv('Honolulu_rental.csv')